In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("new_feauturesExtracted_dataset.csv")
df.head()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,neg,neu,pos,compound,Helpfulness,UnHelpfulness,UserAvgScore,NumUppercase_T,NumUppercase_S,ReviewLength,SummaryLength,Month,Year,numExclamation_S,numExclamation_T
0,1561604,1,4,2.0,1388793600,0.153,0.692,0.155,-0.4040,0.250000,0.250000,2.000000,9,2,97,3,1,2014,1,0
1,516128,5,7,5.0,1033171200,0.000,0.727,0.273,0.9231,0.714286,0.714286,3.083333,6,5,43,5,9,2002,1,0
2,1265775,0,0,4.0,1390521600,0.000,0.767,0.233,0.7901,0.000000,0.000000,4.000000,0,0,25,1,1,2014,0,1
3,484323,3,4,4.0,1136332800,0.030,0.771,0.199,0.9829,0.750000,0.750000,4.500000,14,1,126,3,1,2006,1,3
4,158350,1,2,5.0,1372204800,0.000,0.880,0.120,0.4588,0.500000,0.500000,4.600000,0,1,23,2,6,2013,0,0


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split

predictors = list(df.columns)

predictors.remove("Score")

X = df[predictors]
y=df["Score"]

x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=2, stratify=y)
print('Training set:', x_train.shape, 'Testing set:', x_test.shape)

num_trees = 100
decision_stump = DecisionTreeRegressor(max_depth=1)

boosted = AdaBoostRegressor(base_estimator=decision_stump, n_estimators=num_trees, learning_rate=0.01)
boosted.fit(x_train, y_train);


Training set: (111747, 19) Testing set: (27937, 19)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
def boosting_scores(xtrain, ytrain, xtest, ytest, lam):
  results = dict(num_trees=[], max_depth=[], mse_train=[], mse_test=[])

  for i in range(50,200,10):
    for j in range(1,4):
      decision_stump = DecisionTreeRegressor(max_depth=j)
      boosted = AdaBoostRegressor(base_estimator=decision_stump, n_estimators=i, learning_rate=lam)
      boosted.fit(xtrain, ytrain);
      ypred = boosted.predict(x_train)
      mse_train = mean_squared_error(y_train, ypred)
      results["num_trees"].append(i)
      results["max_depth"].append(j)
      results["mse_train"].append(mse_train)
      y_pred = boosted.predict(x_test)
      mse_test = mean_squared_error(y_test, y_pred)
      results["mse_test"].append(mse_test)

  d = pd.DataFrame(results)
  return d

boost1 = boosting_scores(x_train, y_train, x_test, y_test, 0.01)
boost1.head()

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

In [ ]:
choice = boost1[boost1["mse_test"] == boost1["mse_test"].max()]
choice

,num_trees,max_depth,mse_train,mse_test
0,50,1,0.865198,0.861509


In [ ]:
}#train mse
ypred = boosted.predict(x_train)
mse_train = mean_squared_error(y_train, ypred)
print(mse_train)

0.8548349851258737


In [ ]:
#test mse 
y_pred = boosted.predict(x_test)
mse_test = mean_squared_error(y_test, y_pred)
print(mse_test)

0.848401377859447
